In [1]:
#!pip install pycep_correios

In [2]:
#!pip install geopy

In [3]:
#!pip install pygeocoder

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import pycep_correios
from geopy.geocoders import Nominatim
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np

In [3]:
cwd = os.getcwd()

In [4]:
cwd = os.getcwd()
chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory" : cwd}
chromeOptions.add_experimental_option("prefs",prefs)
chromedriver = cwd + "/chromedriver.exe"
driver = webdriver.Chrome(executable_path=chromedriver, chrome_options=chromeOptions)
driver.get("https://www.google.com.br/maps")

In [5]:
# importação da base de dados com CEP
data = pd.read_excel()

### Coordenadas geográficas de CEP e Endereço Google Maps

In [ ]:
data['Latitude'] = 0.0
data['Longitude'] = 0.0

pesquisar = driver.find_element_by_id('searchboxinput')
enviar = driver.find_element_by_id("searchbox-searchbutton")

for cep in data.CEP:
    coordenadas = 0
    pesquisar.clear()
    pesquisar.send_keys(str(cep))
    enviar.click()
    try:
        time.sleep(3)
        coordenadas = driver.current_url.split("!3d")[1]
        #print(coordenadas)
        latitude = coordenadas.split("!4d")[0]
        longitude = coordenadas.split("!4d")[1]
        data.loc[data.CEP == int(cep),['Latitude']] = latitude
        data.loc[data.CEP == int(cep),['Longitude']] = longitude
    except:
        pass

### Coordenadas geográficas de CEP e Endereço geopy e pycep_correios

In [ ]:

data['Latitude'] = 0.0
data['Longitude'] = 0.0

cep_nd = []
atributos = ['cidade','uf','bairro','end']
addrs = ''
i = 0
for cep_n in data.CEP:
    try:
        for cep in ['0'+ str(cep_n),str(cep_n)]:
            addrs = ''
            try:
                endereco = pycep_correios.consultar_cep(cep)
                check = 0
                for item in atributos:
                    #while check == 0:
                    try:
                        if item == 'bairro':
                            if endereco[item] is None:
                                endereco[item] = 'Centro'
                        addrs = addrs + ' ' + endereco[item]
                        geolocator = Nominatim(user_agent="")
                        location = geolocator.geocode(addrs)
                        data.loc[data.CEP == int(cep),['Latitude']] = location.latitude
                        data.loc[data.CEP == int(cep),['Longitude']] = location.longitude
                        check = 1
                    except:
                        pass
            except:
                pass
    except:
        cep_nd.append(cep)